# __Welcome to the Regression Challenge!__

This notebook provides you some code to setup a Regression problem. There is no optimal solutions, there are many good answers. We only want to see if you know how to tackle the problem. There is a lot of room for your approach to the problem. 

Goal: We want you to predict the number of __sales_per_day__ (= label) with the given dataset.


Short explanation of the dataset: 
- __outlet_id__: The ID of a outlet/market
- __country__: The country in which the outlet is located
- __brand__: "A" or "B"
- __customers_per_day__: The number of customers per day in this outlet 
- __sales_per_day__: The amount of sales for a specific outlet on a specific day
- __currency__: The currency of __sales_per_day__
- __week_id__: Calendar week 
- __weekday__: mon = Monday, tue = Tuesday, ... , sun = Sunday

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn import linear_model
import sklearn.metrics as sklm

%matplotlib inline

In [ ]:
data_path = 'data.csv' # maybe you have to modify this... 
data = pd.read_csv(data_path, sep=';')
data.head(20)

In [ ]:
data.shape  

# Challenge
Now it is your turn. Show us how you are tackeling this problem. You have complete freedom what you do.

## Exploration and Preparation
You have complete freedom what to do. The goal here is to explore the data with e.g. statistics and plots.

In [ ]:
# TODO ... Python coding ... 
data.dtypes

***
**todo**: Columns to be converted or transformed:

- to category: **outlet_id**, **brand**, **country**, **currency**, **weekday**, **week_id**
- add level: weekday

***

Other features are in reasonable data types, and the label **sales_per_day** is numeric.

In [ ]:
# convert data type
def to_categories(data, cols):
    """
    convert column in cols to type category
    """
    for col in cols:
        data[col] = data[col].astype("category")
    
cols = ["outlet_id", "brand", "country", "currency", "weekday", "week_id"]
to_categories(data, cols)


In [ ]:
data.dtypes

In [ ]:
data.info()

In [ ]:
# add level to weekday
def level_categories(data, col, level):
    """
    add level to col in data
    """
    data[col] = data[col].astype("category")
    data[col] = data[col].cat.reorder_categories(level)


weekday_level = ["mon", "tue", "wed", "thu", "fri", "sat", "sun"]
level_categories(data, "weekday", weekday_level)



Until now, the variables in the dataset are in reasonable types.

In [ ]:
label = "sales_per_day"
cat_cols = ["brand", "country", "currency", "outlet_id", "weekday", "week_id"]
num_cols = ["customers_per_day"]

**numeric variables**: Explore the numeric variables with summary statistics

In [ ]:
data.describe()

From the information above, we can see that the label value, sales_per_day, there is a slight difference between the mean and the median values. The median is less than the mean, this indicates that the distribution is right-skewed, which has a tail stretching toward the right.

The large amount of rows with 0 customer cannot be ignored in the dataset, it might be holidays or sundays.

**categorical variable**: Explore the categorical variables with **frequency table**, in order to understand the distributions of them.

In [ ]:
def count_unique(data, cat_cols):
    for col in cat_cols:
        print('\n' + 'Categorical column ' + col)
        print(data[col].value_counts())

count_unique(data, cat_cols)

There are some facts we can derive from the tables above.

1. There is no imbalances in the counts of most of the categories. And for column **currency**, there is a slight difference in the counts. Since there will be few samples for category **CHF**, any statistical property for it will be poorly determined.

2. For variable **outlet_id**, there are a large number of categories. For a dataset of a limited size, it is problematic, because of few samples per category.

***
These are the open points for the following modelling.
***

In [ ]:
def change_to_euro(cols):
    cur = cols[0]
    sales = cols[1]
    if cur == "CHF":
        return sales*0.88 #change CHF to Euro
    else:
        return sales
    
data['sales_per_day'] = data[['currency','sales_per_day']].apply(change_to_euro,axis=1)

- Histograms: to visualize the distribution of the label

In [ ]:
def plot_kde_hist(data, cols, bins = 10, hist = False):
    """
    plot kde of the columns cols of dataframe data. kde is a smoothed version of a histogram. 
    sns is used here to set the style of the grid and plot histogram.
    """
    for col in cols:
        sns.set_style("whitegrid")
        sns.distplot(data[col], bins = bins, rug=False, hist = hist)
        plt.title('Histogram of ' + col)
        plt.xlabel(col) 
        plt.ylabel('Number of rows')
        plt.show()
        
plot_kde_hist(data, num_cols, 40, hist = True)      

There are large amount of observations with sales value of 0. Filter them, and we get a histgram plot with clearer information.

In [ ]:
data_filtered = data[data.sales_per_day > 0]

In [ ]:
plot_kde_hist(data_filtered, num_cols, 40, hist = True)  

print ("Skew is:", data_filtered.sales_per_day.skew())
print("Kurtosis: %f" % data_filtered.sales_per_day.kurt())

The label sales_per_day has a right-skewed, not normal distribution with some multimodal tendency. The skewness will affect the statistical of machine learning  model.

Let's try some feature engineering approach to make it closer to normal, so that it has better performance in modelling.

**transforming numeric variables**: log transformation of sales_per_day

In [ ]:
data_filtered['log_sales'] = np.log(data_filtered['sales_per_day'])
plot_kde_hist(data_filtered, ['log_sales'], 40, hist = True)

The distribution above is more symmetric, but still shows some skew. However, it is still improvement.

Let's dive into the rows which has 0 sales_per_day, we can see that it only happens on sunday.

In [ ]:
data_zero = data[data.sales_per_day == 0]
count_unique(data_zero, cat_cols)

- Scatter plots

To examine the relationship between the features and the label, sales_per_day.

In [ ]:
def plot_scatter(data, cols, col_y = label):
    for col in cols:
        fig = plt.figure(figsize=(7,6)) # define plot area
        ax = fig.gca() # define axis   
        data.plot.scatter(x = col, y = col_y, ax = ax, alpha = 0.01)
        ax.set_title('Scatter plot of ' + col_y + ' vs. ' + col) # Give the plot a main title
        ax.set_xlabel(col) # Set text for the x axis
        ax.set_ylabel(col_y)# Set text for y axis
        plt.show()

cols = ["customers_per_day"]
plot_scatter(data_filtered, cols)    

In [ ]:
numeric_features = data.select_dtypes(include=[np.number])
numeric_features.dtypes

In [ ]:
corr = numeric_features.corr()
corr

### box plot

To examine the relationship between the categorical features and the label, some box plots are
necessary.

In [ ]:
def plot_box(data, cols, col_y = label):
    for col in cols:
        sns.set_style("whitegrid")
        sns.boxplot(col, col_y, data=data)
        plt.xlabel(col) # Set text for the x axis
        plt.ylabel(col_y)# Set text for y axis
        plt.show()

cols = ['brand', 'country', 'currency',  'weekday']
plot_box(data, cols)    

Notice that for some of these cases, there are some noticeable differences between the sales per day by category. For example, for weekday, there are noticeable increase by weekday and 0 on sundays.

## Modelling
Choose a suitable model for the Regression task or do some statistics. 
For training you can use TensorFlow, Keras, Pytorch, scikit-learn or something like that.

We will use scikit-learn package to train a AdaBoosted tree model. 

**reason???**

**one hot encoding**: prepare dummy variables from categorical features

In [ ]:
num_cols

In [ ]:
cat_cols

In [ ]:
data.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder

def cat_encoding(data, IDcols, strCols, catCols):
    le = LabelEncoder()
    
    for IDcol in IDcols:
    #New variable id col
        data['id_'+ IDcol] = le.fit_transform(data[IDcol])
    new_IDcols = [('id_' + i) for i in IDcols]

    for strCol in strCols:
        if strCol in IDcols:
            strCol = 'id_' + strCol
        data[strCol] = le.fit_transform(data[strCol])

    #Dummy Variables, one-hot encoder
    #cat_cols = ['brand', 'country', 'outlet', 'day', 'week']
    cat_cols = catCols + new_IDcols
    data = pd.get_dummies(data, columns = cat_cols)
    return data
    #data.dtypes

save prepared dataset for further reuse.

In [ ]:
ID_cols = ['outlet_id', 'week_id', 'weekday']
cat_cols = ['brand', 'country']
str_cols = ["brand", "country", "weekday"]
data = cat_encoding(data, ID_cols, str_cols, cat_cols)
data.dtypes

In [ ]:
import sys
#Drop unnecessary columns:
data.drop(['currency'],axis=1,inplace=True)

#Export f iles as modified versions:
data.to_csv(sys.path[0]+"/data/prepared.csv",index=False)

## split dataset

In [ ]:
prepared = pd.read_csv(sys.path[0]+"/data/prepared.csv")

In [ ]:
train, test = ms.train_test_split(prepared, test_size = 0.2)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn import metrics

def modelfit(alg, dtrain, dtest, predictors, target):
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain[target])
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    
    #Remember the target had been normalized
    #Sq_train = (dtrain[target])**2
    #Perform cross-validation:
    cv_score = cross_val_score(alg, dtrain[predictors], dtrain[target] , cv=20, scoring='neg_mean_squared_error')
    cv_score = np.sqrt(np.abs(cv_score))
    
    #Print model report:
    print("\nModel Report")
    print("RMSE : %.4g" % np.sqrt(metrics.mean_squared_error((dtrain[target]).values, dtrain_predictions)))
    print("CV Score : Mean - %.4g | Std - %.4g | Min - %.4g | Max - %.4g" % (np.mean(cv_score),np.std(cv_score),np.min(cv_score),np.max(cv_score)))
    
    return alg
    
    

In [ ]:
#Define target and ID columns:
target = 'sales_per_day'
#predicted = 'predicted_sales'
IDcols = ['week_id', 'weekday', 'outlet_id']
predictors = train.columns.drop(['sales_per_day','week_id', 'weekday','outlet_id'])

### Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
LR = LinearRegression(normalize=True)
modelfit(LR, train, test, predictors, target)

### Ridge Regression

In [ ]:
from sklearn.linear_model import Ridge
RR = Ridge(alpha=0.05,normalize=True)
modelfit(RR, train, test, predictors, target)

### Decision Tree Model

In [ ]:
from sklearn.tree import DecisionTreeRegressor
DT = DecisionTreeRegressor(max_depth=15, min_samples_leaf=100)
modelfit(DT, train, test, predictors, target)

### Random Forrest Model

In [ ]:
from sklearn.ensemble import RandomForestRegressor
RF = RandomForestRegressor(n_estimators=10,max_depth=15, min_samples_leaf=100)
modelfit(RF, train, test, predictors, target)

### xgboost model

In [ ]:
from xgboost import XGBRegressor
def xgboost_model(train, test, target, predictors):
    print("\nXgboost Regression")
    my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05)
    my_model.fit(train[predictors], train[target], early_stopping_rounds=5, 
                 eval_set=[(test[predictors], test[target])], verbose=False)

    #Predict training set:
    train_df_predictions = my_model.predict(train[predictors])

    print("RMSE : %.4g" % np.sqrt(metrics.mean_squared_error((train[target]).values, train_df_predictions)))

    return my_model

In [ ]:
xgbmodel = xgboost_model(train, test, target, predictors)

Among all of the regressors, Xgb performs the best with RMSE = 1332.

Do model selection on filtered dataset which removed records with sales_per_day = 0

In [ ]:
data_filtered = cat_encoding(data_filtered, ID_cols, str_cols, cat_cols)
data_filtered.drop(['currency'],axis=1,inplace=True)
train_f, test_f = ms.train_test_split(data_filtered, test_size = 0.2)

In [ ]:
predictors_f = train_f.columns.drop(['sales_per_day','week_id', 'weekday','outlet_id'])

target = 'sales_per_day'
IDcols = ['week_id', 'weekday', 'outlet_id']
filename = '/filtered'

xgbmodel_f = xgboost_model(train_f, test_f, target, predictors_f)

From the model selction on filtered dataset, we can see, the performance improved a lot. Therefore we can use some external public dataset such as sunday or public holiday to predict the sales for them, namely, 0 when it is sunday or public holiday. 

## Evaluation & Outlook
After finishing your training take a look at sample predictions and analyse strengths and weeknesses of your model.

What could be next steps to improve the model?


- harness some public dataset of public holiday or date to predict the sales on sundays or holidays.
- clustering the outlets and do regression on each cluster to get better performance.
- try time series prediction techniques like vector auto regression or RNN such as LSTM on sales_per_day
- Last but not the least, stick to simple model to have better interpretability and prevent overfitting

In [ ]:
import sklearn.metrics as sklm
import math

def print_metrics(y_true, y_predicted):
    ## First compute R^2 and the adjusted R^2
    #r2 = sklm.r2_score(y_true, y_predicted)
    #r2_adj = r2 - (n_parameters - 1)/(y_true.shape[0] - n_parameters) * (1 - r2)
    
    ## Print the usual metrics and the R^2 values
    print('Mean Square Error      = ' + str(sklm.mean_squared_error(y_true, y_predicted)))
    print('Root Mean Square Error = ' + str(math.sqrt(sklm.mean_squared_error(y_true, y_predicted))))
    print('Mean Absolute Error    = ' + str(sklm.mean_absolute_error(y_true, y_predicted)))
    print('Median Absolute Error  = ' + str(sklm.median_absolute_error(y_true, y_predicted)))


In [ ]:
y_score = xgbmodel.predict(test[predictors]) 
print_metrics(test['sales_per_day'], y_score) 

In [ ]:
y_score_f = xgbmodel_f.predict(test_f[predictors_f]) 
print_metrics(test_f['sales_per_day'], y_score_f) 

# Architecture 

Now pretend you need to build a system which runs the model/preciction functionality and should be able to make realtime predictions of the sales per day, everytime new data is generated by the source systems.  
You need to design (a) data pipeline(s) which transfers the data from the sourcesystems to the prediction engine in the format the model/prediction engine can handle it. 

There are the following preconditions: 
    
    - There are 3 source systems: 
        - System A: System which handles __sales_per_day__ 
        - System B: Delivers customers_per_day
        - System C: Stores brand, country, currency and outled_id
        
    - Every system offers an API to call the information
            
    - You want to enable realtime predictions
    
    - You can use any component you like and would use for that use case. Please add to each logical component for your architecture a respective tool. (e.g. for the logical component ETL a tool named Google Dataflow)

Please describe how your architecure would look like with an architecture picture. Describe how the different components will be connected and communicate. Please elaborate why you have choosen a certain logical component + tooling. We expect a moderate level of details within the architecture 

# Send it to us
In the end, please send us: 
    - The ipython notebook 
    - An detailed description of your architecture + an architecture picture (PDF). 